In [1]:
# !pip install PyMongo
# !pip install splinter
# !pip install flask_pymongo
#!pip install webdriver_manager
from bs4 import BeautifulSoup as bs
import requests
import os
import pandas as pd

from splinter import Browser
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import pymongo
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# set up splinter
executable_path = {'executable_path' : ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\wagne\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [3]:
# connect to URL
nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_url)
# set up parser
html = browser.html
soup = bs(html, 'html.parser')

In [4]:
#get most recent news article title and paragraph
#calling 0th element ensures you will get the first title
#go to the inspector pane on the web page to find the class for the header and the paragraph
news_title = soup.find_all('div', class_='content_title')[0].text

news_p = soup.find_all('div', class_='article_teaser_body')[0].text

In [5]:
# print the results
print(news_title)
print("--------------------------")
print(news_p)

Mars Now
--------------------------
The craft’s April 25 flight was conducted at speeds and distances beyond what had ever been previously demonstrated, even in testing on Earth.


In [6]:
#JPL Mars Featured Image 
jpl_url = 'https://www.jpl.nasa.gov'
image_url = "https://www.jpl.nasa.gov/images?search=&category=Mars"
browser.visit(image_url)

html = browser.html
soup_img = bs(html, 'html.parser')

In [17]:
#get relative image path from the search page
relative_url = soup_img.find_all('img')[3]["src"]

print(relative_url)

https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24621.2e16d0ba.fill-400x400-c50.jpg


In [18]:
#add search result to get the featured image path
featured_image_url = image_url + relative_url
print(featured_image_url)

https://www.jpl.nasa.gov/images?search=&category=Marshttps://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24621.2e16d0ba.fill-400x400-c50.jpg


In [ ]:
#Mars facts
# scrape the fact table using pandas
facts_url = 'https://space-facts.com/mars/'
fact_table = pd.read_html(facts_url)
fact_table

In [ ]:
mars_comparison = fact_table[1]
mars_comparison

In [ ]:
mars_comparison_html = mars_comparison.to_html()
mars_comparison_html

In [ ]:
#structure html to be more readable
mars_comparison_html.replace('\n','')
print(mars_comparison_html)

In [ ]:
# mars hemisphere images

astrogeo_url = 'https://astrogeology.usgs.gov'
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(hemi_url)
hemi_html = browser.html
soup_geo = bs(hemi_html, 'html.parser')

In [ ]:
# use beautiful soup to find all data for each hemisphere
hemisphere_path = soup_geo.find('div', class_='collapsible results')
hemisphere_data = hemisphere_path.find_all('div', class_='item')

In [ ]:
#make an empty list for the urls

hemisphere_url_list = []

#loop through the data for each hemisphere to scrape the target data
#title is under h3 in the inspect pane
for h in hemisphere_data:
#     title
    title =h.find('h3').text
#     image link
    hemis = h.find('div', class_= "description")
    hemi_link = hemis.a['href']
    browser.visit(astrogeo_url + hemi_link)    
    html = browser.html
    soup_hemi = bs(html, 'html.parser')   
    img_path = soup_hemi.find('div', class_='downloads')
    img_url = img_path.find('li').a['href']

    # make dictionary for title and url
    hemi_dict = {}
    hemi_dict['title'] = title
    hemi_dict['img_url'] = img_url
    
    hemisphere_url_list.append(hemi_dict)

print(hemisphere_url_list)